# Time travel operations in Hopsworks Feature Store

In this notebook we will introduce time travel operations in Hopsworks Feature Store (HSFS). Currently HSFS supports Apache Hudi (http://hudi.apache.org/) a storage abstraction/library for doing **incremental** data ingestion to a Hopsworks Feature Store.

TLDR; Hudi is a storage abstraction/library build on top of Spark. A Hudi dataset stores data in Parquet files and maintains additional metadata to make upserts efficient. A Hudi ingest job is intended to be run as a streaming ingest job, on an interval such as every 15 minutes, reading deltas from a message-bus like Kafka and ingesting the deltas **incrementally** into a data lake.

![Incremental ETL](./../images/incr_load.png "Incremetal ETL")

## Background

### Motivation

Hudi is an open-source library for doing incremental ingestion of data for large analytical datasets stored on distributed file systems. The library was originally developed at Uber to improve their data latency, but  it is now an Apache project.

The main motivation for Hudi is that it reduces the **data latency** for ingesting large datasets into data lakes. Traditional ETL typically involves taking a snapshot of a production database and doing a full load into a data lake (typically stored on a distributed file system). Using the snapshot approach for ETL is simple since the snapshot is immutable and can be loaded as an atomic unit into the data lake. However, the con of taking this approach to doing data ingestion is that it is *slow*. Even if just a single record have been updated since the last data ingestion, the entire table has to be re-written. If you are working with Big Data (TB or PB size datasets) then this will introduce significant *data latency* (up to 24 hours in Uber's case) and *wasted resources* (majority of the writes when ingesting the snapshot is redundant as most of the records have not been updated since the last ETL step). 

This motivates the use-case for **incremental** data ingestion. Incremental data ingestion means that only deltas/changelogs since the last ingestion are inserted. 

Incremental ingestion lies in-between traditional batch ingestion and the streaming use-case. It can provide data latency as low as *minutes* for petabyte-scale datasets. The incremental mode for processing introduces new trade-offs compared to streaming and batch. It has lower data latency than traditional batch processing, but a slightly higher latency than stream processing. Why not go full-streaming instead of the incremental processing? It boils down to your requirements and trade-offs. If you need data latency in the order of seconds, then you have to use stream processing (e.g fraud detection). However if your business can do with data latency in the order of say 5 minutes (applications which are fine with this latency could be feature engineering pipelines, building dashboards, or doing near-real-time analytics), then incremental processing really shines. 

With incremental processing, you process data in *mini-batches* and run the spark job frequently, every 15 minutes or so. By using mini-batches rather than record-by-record streaming, the incremental model makes better use of resources and makes it easier to do complex processing and joins which are more suited for the batch-style of processing rather than stream-processing.

![Near Real Time](./../images/near_real_time.jpg "Near Real Time")

If the data is immutable by design, incremental processing can be done without any additional ingestion library, just use the *append* primitive supported in HDFS through some HDFS client, such as Spark, e.g:

```scala
newRecordsDf = (...)
newRecordsDf.write.format("hive").mode("append").insertInto(tableName)
```

Unfortunately, data is rarely immutable in practice. A bank transaction might be reverted, a customer might change his or her home adress, and a customer review might be updated, to give a few examples. This is where Hudi comes into the picture. Hudi stands for `Hadoop Upserts anD Incrementals` and brings two new primitives for data engineering on distributed file systems (in addition to append/read):

- `Upsert`: the ability to do insertions (appends) and updates efficiently. 
- `Incremental reads`: the ability to read datasets incrementally using the notion of "commits".

![Upserts](./../images/upsert_illustration.png "Upserts")

Lets consider the process of updating a single record in a data lake of Parquet files stored on a distributed file system. Without using Hudi, this would entail scanning the entire dataset to find the record in order to do the update and then rewrite the entire dataset: 

```scala
updatedRecordsDf = (...)
updatedRecordsDf.write.format("hive").mode("overwrite").insertInto(tableName) 
```

This does not scale and HDFS/Parquet is not designed for this use-case. With Hudi, the upsert operation is a first-class primitive in the ingestion framework and it is optimized to be fast using index-lookups and atomic updates. We will see how we can use Hudi for this purpose later on in the notebook, but essentially it is as simple as :

```scala
updatedRecordsDf = (...)
upsertDf.write.format("org.apache.hudi")
              .option("hoodie.datasource.write.operation", "upsert")
              ...
```

### What is Hudi

Hudi is a Spark library that is intended to be run as a streaming ingest job, and ingests data as mini-batches (typically on the order of one to two minutes). A Hudi job generally reads delta-updates from a message-bus like Kafka, and upserts them into a data lake stored on a distributed file system. By maintaining bloom indexes and commit logs, Hudi provide ACID transactions, time-travel and scalable upserts.

![Hudi Dataset](./../images/hudi_dataset.png "Hudi Dataset")

### How Hopsworks Feature Store time travel operations can be used for ML and Feature Pipelines

Hudi is integrated in the Hopsworks Feature Store for doing incremental feature computation and for point-in-time correctness and backfilling of feature data.

![Incremental Feature Engineering](./../images/featurestore_incremental_pull.png "Incremetal Feature Engineering")

In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
22,application_1604621247318_0036,spark,idle,Link,Link


SparkSession available as 'spark'.
res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@26daa88


## Examples

### Create HUDI time travel enabled feature group and Bulk Insert Sample Dataset

For this demo we will use small sample of the Agarwal Generator that is a widely used dataset. It contains the hypothetical data of people applying for a loan. `Rakesh Agrawal, Tomasz Imielinksi, and Arun Swami, "Database Mining: A Performance Perspective", IEEE Transactions on Knowledge and Data Engineering, 5(6), December 1993. <br/><br/>`

##### For simplicity of demo purposes we will split Agarwal dataset into 3 freature groups and manualy create datasets: 
* `economy_fg` with customer id, salary, loan, value of house, age of house, commission and type of car features; 
* `demographic_fg` with customer id, age, education level, zip code,
* `class_fg` which will contain labels wether loan was approved `class B` or rejected `class A`.

### Importing necessary libraries 

In [2]:
import com.logicalclocks.hsfs._
import scala.collection.JavaConversions._
import collection.JavaConverters._

import org.apache.spark.sql.{ DataFrame, Row }
import org.apache.spark.sql.catalyst.expressions.GenericRow
import org.apache.spark.sql.types._

import java.sql.Date
import java.sql.Timestamp

val connection = HopsworksConnection.builder().build();
val fs = connection.getFeatureStore();

import com.logicalclocks.hsfs._
import scala.collection.JavaConversions._
import collection.JavaConverters._
import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.catalyst.expressions.GenericRow
import org.apache.spark.sql.types._
import java.sql.Date
import java.sql.Timestamp
connection: com.logicalclocks.hsfs.HopsworksConnection = com.logicalclocks.hsfs.HopsworksConnection@1af1cc43
fs: com.logicalclocks.hsfs.FeatureStore = FeatureStore{id=67, name='demo_fs_meb10000_featurestore', projectId=119, featureGroupApi=com.logicalclocks.hsfs.metadata.FeatureGroupApi@68518159}


In [44]:
val economyFgSchema = 
 scala.collection.immutable.List(
  StructField("id", IntegerType, true),
  StructField("salary", FloatType, true),
  StructField("commission", FloatType, true),
  StructField("car", StringType, true), 
  StructField("hvalue", FloatType, true),      
  StructField("hyears", IntegerType, true),     
  StructField("loan", FloatType, true),
  StructField("year", IntegerType, true)          
)

val demographicFgSchema = 
 scala.collection.immutable.List(
  StructField("id", IntegerType, true),
  StructField("age", IntegerType, true),
  StructField("elevel", StringType, true),   
  StructField("zipcode", StringType, true) 
)

val classFgSchema = 
 scala.collection.immutable.List(
  StructField("id", IntegerType, true),
  StructField("class", StringType, true),
  StructField("year", IntegerType, true)          
)

economyFgSchema: List[org.apache.spark.sql.types.StructField] = List(StructField(id,IntegerType,true), StructField(salary,FloatType,true), StructField(commission,FloatType,true), StructField(car,StringType,true), StructField(hvalue,FloatType,true), StructField(hyears,IntegerType,true), StructField(loan,FloatType,true), StructField(year,IntegerType,true))
demographicFgSchema: List[org.apache.spark.sql.types.StructField] = List(StructField(id,IntegerType,true), StructField(age,IntegerType,true), StructField(elevel,StringType,true), StructField(zipcode,StringType,true))
classFgSchema: List[org.apache.spark.sql.types.StructField] = List(StructField(id,IntegerType,true), StructField(class,StringType,true), StructField(year,IntegerType,true))


### Create spark dataframes for each Feature groups

In [45]:
val economyBulkInsertData = Seq(
    Row(1, 110499.73f, 0.0f,  "car15",  235000.0f, 30, 354724.18f, 2020),
    Row(2, 140893.77f, 0.0f,  "car20",  135000.0f, 2, 395015.33f, 2020),
    Row(3, 119159.65f, 0.0f,  "car1", 145000.0f, 22, 122025.08f, 2020),
    Row(4, 20000.0f, 52593.63f, "car9", 185000.0f, 30, 99629.62f, 2020)
)

val economyBulkInsertDf = spark.createDataFrame(
    spark.sparkContext.parallelize(economyBulkInsertData),
    StructType(economyFgSchema)
)

economyBulkInsertData: Seq[org.apache.spark.sql.Row] = List([1,110499.73,0.0,car15,235000.0,30,354724.2,2020], [2,140893.77,0.0,car20,135000.0,2,395015.34,2020], [3,119159.65,0.0,car1,145000.0,22,122025.08,2020], [4,20000.0,52593.63,car9,185000.0,30,99629.62,2020])
economyBulkInsertDf: org.apache.spark.sql.DataFrame = [id: int, salary: float ... 6 more fields]


In [46]:
economyBulkInsertDf.show()

+---+---------+----------+-----+--------+------+---------+----+
| id|   salary|commission|  car|  hvalue|hyears|     loan|year|
+---+---------+----------+-----+--------+------+---------+----+
|  1|110499.73|       0.0|car15|235000.0|    30| 354724.2|2020|
|  2|140893.77|       0.0|car20|135000.0|     2|395015.34|2020|
|  3|119159.65|       0.0| car1|145000.0|    22|122025.08|2020|
|  4|  20000.0|  52593.63| car9|185000.0|    30| 99629.62|2020|
+---+---------+----------+-----+--------+------+---------+----+



In [47]:
val demographicBulkInsertData = Seq(
    Row(1, 54, "level3", "zipcode5"),
    Row(2, 44, "level4", "zipcode8"),
    Row(3, 49, "level2", "zipcode4"),
    Row(4, 56, "level0", "zipcode2")
)

val demographicBulkInsertDf = spark.createDataFrame(
    spark.sparkContext.parallelize(demographicBulkInsertData),
    StructType(demographicFgSchema)
)

demographicBulkInsertData: Seq[org.apache.spark.sql.Row] = List([1,54,level3,zipcode5], [2,44,level4,zipcode8], [3,49,level2,zipcode4], [4,56,level0,zipcode2])
demographicBulkInsertDf: org.apache.spark.sql.DataFrame = [id: int, age: int ... 2 more fields]


In [48]:
demographicBulkInsertDf.show()

+---+---+------+--------+
| id|age|elevel| zipcode|
+---+---+------+--------+
|  1| 54|level3|zipcode5|
|  2| 44|level4|zipcode8|
|  3| 49|level2|zipcode4|
|  4| 56|level0|zipcode2|
+---+---+------+--------+



In [49]:
val classBulkInsertData = Seq(
    Row(1, "groupB", 2020),
    Row(2, "groupB", 2020),
    Row(3, "groupB", 2020),
    Row(4, "groupB", 2020)
) 

val classBulkInsertDf = spark.createDataFrame(
    spark.sparkContext.parallelize(classBulkInsertData),
    StructType(classFgSchema)
)

classBulkInsertData: Seq[org.apache.spark.sql.Row] = List([1,groupB,2020], [2,groupB,2020], [3,groupB,2020], [4,groupB,2020])
classBulkInsertDf: org.apache.spark.sql.DataFrame = [id: int, class: string ... 1 more field]


In [50]:
classBulkInsertDf.show()

+---+------+----+
| id| class|year|
+---+------+----+
|  1|groupB|2020|
|  2|groupB|2020|
|  3|groupB|2020|
|  4|groupB|2020|
+---+------+----+



### Create feature groups 

Now we will create each feature group and enable time travel format `TimeTravelFormat.HUDI`. In Hopsworks Feature Store `primary` and `partition` keys are required to be privided for HUDI enabled feature groups.

In [51]:
val economyFg = (fs.createFeatureGroup().
                name("economy_fg").
                description("Hudi Household Economy Feature Group").
                version(1).
                primaryKeys(Seq("id")).
                partitionKeys(Seq("year")).
                timeTravelFormat(TimeTravelFormat.HUDI).
                build())

economyFg: com.logicalclocks.hsfs.FeatureGroup = com.logicalclocks.hsfs.FeatureGroup@5daf72d7


In [52]:
val demographyFg = (fs.createFeatureGroup().
                    name("demography_fg").
                    description("Hudi Demographic Feature Group").
                    version(1).
                    primaryKeys(Seq("id")).
                    partitionKeys(Seq("zipcode")).
                    timeTravelFormat(TimeTravelFormat.HUDI).
                    build())

demographyFg: com.logicalclocks.hsfs.FeatureGroup = com.logicalclocks.hsfs.FeatureGroup@76fc27b5


In [53]:
val classFg = (fs.createFeatureGroup().
                name("class_fg").
                description("Hudi Class Feature Group").
                version(1).
                primaryKeys(Seq("id")).
                partitionKeys(Seq("year")).
                timeTravelFormat(TimeTravelFormat.HUDI).
                build())

classFg: com.logicalclocks.hsfs.FeatureGroup = com.logicalclocks.hsfs.FeatureGroup@680629ee


### Bulk insert data into the feature group
Since we have not yet saved any data into newly created feature groups we will use Apache hudi terminology and `Bulk Insert` data. In HSFS its just issuing `save` method.  

In [54]:
economyFg.save(economyBulkInsertDf)

In [55]:
demographyFg.save(demographicBulkInsertDf)

In [56]:
classFg.save(classBulkInsertDf)

#### Hopsworks Feature Store Commits

If you thoroughly followed this demo you probably noticed that Hopsworks Feature Store uses Apache Hudi as its time travel engine. Hudi introduces the notion of `commits` which means that it supports certain properties of traditional databases such as single-table transactions, snapshot isolation, atomic upserts and savepoints for data recovery. If an ingestion fails for some reason, no partial results will be written rather the ingestion will be roll-backed. The commit is implemented using atomic `mv` operation in HDFS. 

Currently, feature groups that we created contain only a single commit each as we've just done a single bulk-insert. Lets explore time line of `economyFg`:

In [57]:
for ((k,v) <- economyFg.commitDetails()){
    println (k,v)
}

(1604795225000,{committedOn=20201108002705, rowsUpdated=0, rowsDeleted=0, rowsInserted=4})


### Inspect results

In [58]:
economyFg.read().show()

+---+---------+----------+-----+--------+------+---------+----+
| id|   salary|commission|  car|  hvalue|hyears|     loan|year|
+---+---------+----------+-----+--------+------+---------+----+
|  2|140893.77|       0.0|car20|135000.0|     2|395015.34|2020|
|  1|110499.73|       0.0|car15|235000.0|    30| 354724.2|2020|
|  4|  20000.0|  52593.63| car9|185000.0|    30| 99629.62|2020|
|  3|119159.65|       0.0| car1|145000.0|    22|122025.08|2020|
+---+---------+----------+-----+--------+------+---------+----+



In [59]:
demographyFg.read().show()

+---+---+------+--------+
| id|age|elevel| zipcode|
+---+---+------+--------+
|  3| 49|level2|zipcode4|
|  2| 44|level4|zipcode8|
|  4| 56|level0|zipcode2|
|  1| 54|level3|zipcode5|
+---+---+------+--------+



In [60]:
classFg.read().show()

+---+------+----+
| id| class|year|
+---+------+----+
|  2|groupB|2020|
|  3|groupB|2020|
|  4|groupB|2020|
|  1|groupB|2020|
+---+------+----+



### Upsert new data into a Feature Group

So far we have not done anything time travel special, we simply did a regular bulk-insert of some data into a Hudi enabled feature group. We could have done the same thing using just regular None Hudi enabled Feature group. However now we will look into how we can do upserts, and how Hopsworks Feature store enables us to do this efficiently.

#### Generate Sample Upserts Data

In [61]:
val economyUpsertData = Seq(
    Row(1, 120499.73f, 0.0f, "car17", 205000.0f, 30, 564724.18f, 2020),    //update
    Row(2, 160893.77f, 0.0f, "car10", 179000.0f, 2, 455015.33f, 2020),     //update
    Row(5, 93956.32f, 0.0f, "car15",  135000.0f, 1, 458679.82f, 2020),     //insert
    Row(6, 41365.43f, 52809.15f, "car7", 135000.0f, 19, 216839.71f, 2020), //insert
    Row(7, 94805.61f, 0.0f, "car17", 135000.0f, 23, 233216.07f, 2020)      //insert
)

val economyUpsertDf = spark.createDataFrame(
  spark.sparkContext.parallelize(economyUpsertData),
  StructType(economyFgSchema)
)

economyUpsertDf.show(5)

economyUpsertData: Seq[org.apache.spark.sql.Row] = List([1,120499.73,0.0,car17,205000.0,30,564724.2,2020], [2,160893.77,0.0,car10,179000.0,2,455015.34,2020], [5,93956.32,0.0,car15,135000.0,1,458679.8,2020], [6,41365.43,52809.15,car7,135000.0,19,216839.7,2020], [7,94805.61,0.0,car17,135000.0,23,233216.06,2020])
economyUpsertDf: org.apache.spark.sql.DataFrame = [id: int, salary: float ... 6 more fields]
+---+---------+----------+-----+--------+------+---------+----+
| id|   salary|commission|  car|  hvalue|hyears|     loan|year|
+---+---------+----------+-----+--------+------+---------+----+
|  1|120499.73|       0.0|car17|205000.0|    30| 564724.2|2020|
|  2|160893.77|       0.0|car10|179000.0|     2|455015.34|2020|
|  5| 93956.32|       0.0|car15|135000.0|     1| 458679.8|2020|
|  6| 41365.43|  52809.15| car7|135000.0|    19| 216839.7|2020|
|  7| 94805.61|       0.0|car17|135000.0|    23|233216.06|2020|
+---+---------+----------+-----+--------+------+---------+----+



In [62]:
val demographicUpsertData = Seq(
    Row(2, 44, "level1", "zipcode8"),     //update
    Row(5, 59, "level1", "zipcode2"),     //insert
    Row(6, 71, "level2", "zipcode3"),     //insert
    Row(7, 32, "level1", "zipcode2")      //insert
)

val demographicUpsertDf = spark.createDataFrame(
    spark.sparkContext.parallelize(demographicUpsertData),
    StructType(demographicFgSchema)
)

demographicUpsertDf.show()

demographicUpsertData: Seq[org.apache.spark.sql.Row] = List([2,44,level1,zipcode8], [5,59,level1,zipcode2], [6,71,level2,zipcode3], [7,32,level1,zipcode2])
demographicUpsertDf: org.apache.spark.sql.DataFrame = [id: int, age: int ... 2 more fields]
+---+---+------+--------+
| id|age|elevel| zipcode|
+---+---+------+--------+
|  2| 44|level1|zipcode8|
|  5| 59|level1|zipcode2|
|  6| 71|level2|zipcode3|
|  7| 32|level1|zipcode2|
+---+---+------+--------+



In [75]:
val classUpsertData = Seq(
    Row(1, "groupA", 2020), //update
    Row(5, "groupA", 2020), //insert
    Row(6, "groupA", 2020), //insert
    Row(7, "groupA", 2020)  //insert
) 

val classUpsertDf = spark.createDataFrame(
    spark.sparkContext.parallelize(classUpsertData),
    StructType(classFgSchema)
)

classUpsertDf.show()

classUpsertData: Seq[org.apache.spark.sql.Row] = List([1,groupA,2020], [5,groupA,2020], [6,groupA,2020], [7,groupA,2020])
classUpsertDf: org.apache.spark.sql.DataFrame = [id: int, class: string ... 1 more field]
+---+------+----+
| id| class|year|
+---+------+----+
|  1|groupA|2020|
|  5|groupA|2020|
|  6|groupA|2020|
|  7|groupA|2020|
+---+------+----+



#### Make the Upsert using Hopsworks Feature Store API
In Hopsworks Feature Store issuing `insert` method on Apache Hudi enabled feature group will by default perform `UPSERT` operation which means to either insert a new row, or on the basis of `parimary` and `partition` keys or update already existing one.

In [64]:
economyFg.insert(economyUpsertDf)

In [65]:
demographyFg.insert(demographicUpsertDf)

In [76]:
classFg.insert(classUpsertDf)

#### Inspect the results

Notice that although Hudi enabled Feature group stores the old value of the records from the previous commit, when you query it will only return the values of the latest commit.

In [67]:
economyFg.read().show()

+---+---------+----------+-----+--------+------+---------+----+
| id|   salary|commission|  car|  hvalue|hyears|     loan|year|
+---+---------+----------+-----+--------+------+---------+----+
|  1|120499.73|       0.0|car17|205000.0|    30| 564724.2|2020|
|  5| 93956.32|       0.0|car15|135000.0|     1| 458679.8|2020|
|  6| 41365.43|  52809.15| car7|135000.0|    19| 216839.7|2020|
|  7| 94805.61|       0.0|car17|135000.0|    23|233216.06|2020|
|  2|160893.77|       0.0|car10|179000.0|     2|455015.34|2020|
|  4|  20000.0|  52593.63| car9|185000.0|    30| 99629.62|2020|
|  3|119159.65|       0.0| car1|145000.0|    22|122025.08|2020|
+---+---------+----------+-----+--------+------+---------+----+



In [68]:
demographyFg.read().show()

+---+---+------+--------+
| id|age|elevel| zipcode|
+---+---+------+--------+
|  2| 44|level1|zipcode8|
|  6| 71|level2|zipcode3|
|  3| 49|level2|zipcode4|
|  1| 54|level3|zipcode5|
|  4| 56|level0|zipcode2|
|  5| 59|level1|zipcode2|
|  7| 32|level1|zipcode2|
+---+---+------+--------+



In [77]:
classFg.read.show()

+---+------+----+
| id| class|year|
+---+------+----+
|  1|groupA|2020|
|  5|groupA|2020|
|  6|groupA|2020|
|  7|groupA|2020|
|  2|groupB|2020|
|  3|groupB|2020|
|  4|groupB|2020|
+---+------+----+



#### Inspect the updated commit timeline of `economyFg`

In [70]:
for ((k,v) <- economyFg.commitDetails()){
    println (k,v)
}

(1604795370000,{committedOn=20201108002930, rowsUpdated=2, rowsDeleted=0, rowsInserted=3})
(1604795225000,{committedOn=20201108002705, rowsUpdated=0, rowsDeleted=0, rowsInserted=4})


In [ ]:
for ((k,v) <- demographyFg.commitDetails()){
    println (k,v)
}

In [72]:
for ((k,v) <- classFg.commitDetails()){
    println (k,v)
}

(1604795324000,{committedOn=20201108002844, rowsUpdated=0, rowsDeleted=0, rowsInserted=4})


### Lets make one more commit to better demostrate time travel capabilities of Hopsworks Feature Store

In [79]:
val economyUpsertData = Seq(    
    Row(8, 64410.62f, 39884.39f, "car20",  125000.0f, 6, 350707.38f, 2020), //insert
    Row(9, 128298.82f, 0.0f, "car19",  135000.0f, 12, 20768.06f, 2020),     //insert
    Row(10,100806.92f, 0.0f, "car8", 135000.0f, 6, 293106.65f, 2020)        //insert   
    
)

val economyUpsertDf = spark.createDataFrame(
  spark.sparkContext.parallelize(economyUpsertData),
  StructType(economyFgSchema)
)

economyUpsertDf.show()

economyUpsertData: Seq[org.apache.spark.sql.Row] = List([8,64410.62,39884.39,car20,125000.0,6,350707.38,2020], [9,128298.82,0.0,car19,135000.0,12,20768.06,2020], [10,100806.92,0.0,car8,135000.0,6,293106.66,2020])
economyUpsertDf: org.apache.spark.sql.DataFrame = [id: int, salary: float ... 6 more fields]
+---+---------+----------+-----+--------+------+---------+----+
| id|   salary|commission|  car|  hvalue|hyears|     loan|year|
+---+---------+----------+-----+--------+------+---------+----+
|  8| 64410.62|  39884.39|car20|125000.0|     6|350707.38|2020|
|  9|128298.82|       0.0|car19|135000.0|    12| 20768.06|2020|
| 10|100806.92|       0.0| car8|135000.0|     6|293106.66|2020|
+---+---------+----------+-----+--------+------+---------+----+



In [80]:
val demographicUpsertData = Seq(    
    Row(8, 33, "level2", "zipcode1"),     //insert
    Row(9, 32, "level1", "zipcode3"),     //insert
    Row(10, 58, "level2", "zipcode5")     //insert    
)

val demographicUpsertDf = spark.createDataFrame(
    spark.sparkContext.parallelize(demographicUpsertData),
    StructType(demographicFgSchema)
)

demographicUpsertDf.show()

demographicUpsertData: Seq[org.apache.spark.sql.Row] = List([8,33,level2,zipcode1], [9,32,level1,zipcode3], [10,58,level2,zipcode5])
demographicUpsertDf: org.apache.spark.sql.DataFrame = [id: int, age: int ... 2 more fields]
+---+---+------+--------+
| id|age|elevel| zipcode|
+---+---+------+--------+
|  8| 33|level2|zipcode1|
|  9| 32|level1|zipcode3|
| 10| 58|level2|zipcode5|
+---+---+------+--------+



In [81]:
val classUpsertData = Seq(
    Row(8, "groupA", 2020), //insert
    Row(9, "groupA", 2020), //insert
    Row(10, "groupB", 2020) //insert    
) 

val classUpsertDf = spark.createDataFrame(
    spark.sparkContext.parallelize(classUpsertData),
    StructType(classFgSchema)
)

classUpsertDf.show()

classUpsertData: Seq[org.apache.spark.sql.Row] = List([8,groupA,2020], [9,groupA,2020], [10,groupB,2020])
classUpsertDf: org.apache.spark.sql.DataFrame = [id: int, class: string ... 1 more field]
+---+------+----+
| id| class|year|
+---+------+----+
|  8|groupA|2020|
|  9|groupA|2020|
| 10|groupB|2020|
+---+------+----+



In [82]:
economyFg.insert(economyUpsertDf)

In [ ]:
demographyFg.insert(demographicUpsertDf)

In [ ]:
classFg.insert(classUpsertDf)

### Time Travel Queries
When `read` method is issued on `FeatureGroup` object, whithout any aparameters, most recent view of the Feature group will be returned. 

In [7]:
economyFg.read().show()

+---------+---+------+----------+-----+---------+--------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|
+---------+---+------+----------+-----+---------+--------+----+
| 564724.2|  1|    30|       0.0|car17|120499.73|205000.0|2020|
| 458679.8|  5|     1|       0.0|car15| 93956.32|135000.0|2020|
| 216839.7|  6|    19|  52809.15| car7| 41365.43|135000.0|2020|
|233216.06|  7|    23|       0.0|car17| 94805.61|135000.0|2020|
|350707.38|  8|     6|  39884.39|car20| 64410.62|125000.0|2020|
| 20768.06|  9|    12|       0.0|car19|128298.82|135000.0|2020|
|293106.66| 10|     6|       0.0| car8|100806.92|135000.0|2020|
|455015.34|  2|     2|       0.0|car10|160893.77|179000.0|2020|
| 99629.62|  4|    30|  52593.63| car9|  20000.0|185000.0|2020|
|122025.08|  3|    22|       0.0| car1|119159.65|145000.0|2020|
+---------+---+------+----------+-----+---------+--------+----+



Using the timeline metadata we can inspect the value of a table at a specific point in time, as well as pull changes incrementally.

In [8]:
val commitTimeline = economyFg.commitDetails()
for ((k,v) <- commitTimeline){
    println (k,v)
}

commitTimeline: java.util.Map[String,java.util.Map[String,String]] = {1604795370000={committedOn=20201108002930, rowsUpdated=2, rowsDeleted=0, rowsInserted=3}, 1604795225000={committedOn=20201108002705, rowsUpdated=0, rowsDeleted=0, rowsInserted=4}, 1604797033000={committedOn=20201108005713, rowsUpdated=0, rowsDeleted=0, rowsInserted=3}}
(1604795370000,{committedOn=20201108002930, rowsUpdated=2, rowsDeleted=0, rowsInserted=3})
(1604795225000,{committedOn=20201108002705, rowsUpdated=0, rowsDeleted=0, rowsInserted=4})
(1604797033000,{committedOn=20201108005713, rowsUpdated=0, rowsDeleted=0, rowsInserted=3})


In [10]:
// pull 1st commit
economyFg.read("2020-11-08 00:27:05").show()

+---------+---+------+----------+-----+---------+--------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|
+---------+---+------+----------+-----+---------+--------+----+
|395015.34|  2|     2|       0.0|car20|140893.77|135000.0|2020|
| 354724.2|  1|    30|       0.0|car15|110499.73|235000.0|2020|
| 99629.62|  4|    30|  52593.63| car9|  20000.0|185000.0|2020|
|122025.08|  3|    22|       0.0| car1|119159.65|145000.0|2020|
+---------+---+------+----------+-----+---------+--------+----+



In [11]:
// pull 2nd commit
economyFg.read("2020-11-08 00:29:30").show()

+---------+---+------+----------+-----+---------+--------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|
+---------+---+------+----------+-----+---------+--------+----+
| 564724.2|  1|    30|       0.0|car17|120499.73|205000.0|2020|
| 458679.8|  5|     1|       0.0|car15| 93956.32|135000.0|2020|
| 216839.7|  6|    19|  52809.15| car7| 41365.43|135000.0|2020|
|233216.06|  7|    23|       0.0|car17| 94805.61|135000.0|2020|
|455015.34|  2|     2|       0.0|car10|160893.77|179000.0|2020|
| 99629.62|  4|    30|  52593.63| car9|  20000.0|185000.0|2020|
|122025.08|  3|    22|       0.0| car1|119159.65|145000.0|2020|
+---------+---+------+----------+-----+---------+--------+----+



In [13]:
// pull 3rd commit
economyFg.read("2020-11-08 00:57:13").show()

+---------+---+------+----------+-----+---------+--------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|
+---------+---+------+----------+-----+---------+--------+----+
| 564724.2|  1|    30|       0.0|car17|120499.73|205000.0|2020|
| 458679.8|  5|     1|       0.0|car15| 93956.32|135000.0|2020|
| 216839.7|  6|    19|  52809.15| car7| 41365.43|135000.0|2020|
|233216.06|  7|    23|       0.0|car17| 94805.61|135000.0|2020|
|350707.38|  8|     6|  39884.39|car20| 64410.62|125000.0|2020|
| 20768.06|  9|    12|       0.0|car19|128298.82|135000.0|2020|
|293106.66| 10|     6|       0.0| car8|100806.92|135000.0|2020|
|455015.34|  2|     2|       0.0|car10|160893.77|179000.0|2020|
| 99629.62|  4|    30|  52593.63| car9|  20000.0|185000.0|2020|
|122025.08|  3|    22|       0.0| car1|119159.65|145000.0|2020|
+---------+---+------+----------+-----+---------+--------+----+



### Hopsworks Feature Store also provides a method for incremental reads:

In [15]:
// Pull changes that happened between the first and second commits
economyFg.readChanges("2020-11-08 00:27:05", "2020-11-08 00:29:30").show()

+---------+---+------+----------+-----+---------+--------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|
+---------+---+------+----------+-----+---------+--------+----+
| 564724.2|  1|    30|       0.0|car17|120499.73|205000.0|2020|
| 458679.8|  5|     1|       0.0|car15| 93956.32|135000.0|2020|
| 216839.7|  6|    19|  52809.15| car7| 41365.43|135000.0|2020|
|233216.06|  7|    23|       0.0|car17| 94805.61|135000.0|2020|
|455015.34|  2|     2|       0.0|car10|160893.77|179000.0|2020|
+---------+---+------+----------+-----+---------+--------+----+



In [16]:
// Pull changes that happened between the second and third commits 
economyFg.readChanges("2020-11-08 00:29:30", "2020-11-08 00:57:13").show()

+---------+---+------+----------+-----+---------+--------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|
+---------+---+------+----------+-----+---------+--------+----+
|350707.38|  8|     6|  39884.39|car20| 64410.62|125000.0|2020|
| 20768.06|  9|    12|       0.0|car19|128298.82|135000.0|2020|
|293106.66| 10|     6|       0.0| car8|100806.92|135000.0|2020|
+---------+---+------+----------+-----+---------+--------+----+



### Join Feature groups that correspond to specific point in time
If we are interetsted to join Feature groups all of them correspong to one specific point in time then we can issue `asOf` method on join `Query` object.  

In [19]:
val joined_features = ((economyFg.selectAll())
                   .join(demographyFg.selectAll(), Seq("id"), JoinType.INNER)
                   .join(classFg.selectAll(), Seq("id"), JoinType.INNER)
                   .asOf("2020-11-08 01:00:00"))  

joined_features: com.logicalclocks.hsfs.metadata.Query = SELECT `fg0`.`loan`, `fg0`.`id`, `fg0`.`hyears`, `fg0`.`commission`, `fg0`.`car`, `fg0`.`salary`, `fg0`.`hvalue`, `fg0`.`year`, `fg1`.`elevel`, `fg1`.`age`, `fg1`.`id`, `fg1`.`zipcode`, `fg2`.`id`, `fg2`.`class`, `fg2`.`year` FROM `fg0` `fg0` INNER JOIN `fg1` `fg1` ON `fg0`.`id` = `fg1`.`id` INNER JOIN `fg2` `fg2` ON `fg0`.`id` = `fg2`.`id`


In [21]:
joined_features.read().show()

+---------+---+------+----------+-----+---------+--------+----+------+---+---+--------+---+------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|elevel|age| id| zipcode| id| class|year|
+---------+---+------+----------+-----+---------+--------+----+------+---+---+--------+---+------+----+
| 564724.2|  1|    30|       0.0|car17|120499.73|205000.0|2020|level3| 54|  1|zipcode5|  1|groupA|2020|
| 216839.7|  6|    19|  52809.15| car7| 41365.43|135000.0|2020|level2| 71|  6|zipcode3|  6|groupA|2020|
|122025.08|  3|    22|       0.0| car1|119159.65|145000.0|2020|level2| 49|  3|zipcode4|  3|groupB|2020|
| 458679.8|  5|     1|       0.0|car15| 93956.32|135000.0|2020|level1| 59|  5|zipcode2|  5|groupA|2020|
| 99629.62|  4|    30|  52593.63| car9|  20000.0|185000.0|2020|level0| 56|  4|zipcode2|  4|groupB|2020|
|233216.06|  7|    23|       0.0|car17| 94805.61|135000.0|2020|level1| 32|  7|zipcode2|  7|groupA|2020|
|455015.34|  2|     2|       0.0|car10|160893.77|179000.0|2020|l

### Join Feature groups that correspond to different points in time
Hopswork Feature store also provides functionality to join Feature groups that correspond to different points in time. 

In [23]:
val economyFgQuery = economyFg.selectAll().asOf("2020-11-08 00:57:13")
val demographyFgQuery = demographyFg.selectAll().asOf("2020-11-08-01:01:17")
val classFgQuery =  classFg.selectAll().asOf("2020-11-08-01:01:17")

economyFgQuery: com.logicalclocks.hsfs.metadata.Query = SELECT `fg0`.`loan`, `fg0`.`id`, `fg0`.`hyears`, `fg0`.`commission`, `fg0`.`car`, `fg0`.`salary`, `fg0`.`hvalue`, `fg0`.`year` FROM `fg0` `fg0`
demographyFgQuery: com.logicalclocks.hsfs.metadata.Query = SELECT `fg0`.`elevel`, `fg0`.`age`, `fg0`.`id`, `fg0`.`zipcode` FROM `fg0` `fg0`
classFgQuery: com.logicalclocks.hsfs.metadata.Query = SELECT `fg0`.`id`, `fg0`.`class`, `fg0`.`year` FROM `fg0` `fg0`


In [24]:
val joined_features = economyFgQuery.join(demographyFgQuery, Seq("id"), JoinType.INNER).join(classFgQuery, Seq("id"), JoinType.INNER)

joined_features: com.logicalclocks.hsfs.metadata.Query = SELECT `fg0`.`loan`, `fg0`.`id`, `fg0`.`hyears`, `fg0`.`commission`, `fg0`.`car`, `fg0`.`salary`, `fg0`.`hvalue`, `fg0`.`year`, `fg1`.`elevel`, `fg1`.`age`, `fg1`.`id`, `fg1`.`zipcode`, `fg2`.`id`, `fg2`.`class`, `fg2`.`year` FROM `fg0` `fg0` INNER JOIN `fg1` `fg1` ON `fg0`.`id` = `fg1`.`id` INNER JOIN `fg2` `fg2` ON `fg0`.`id` = `fg2`.`id`


In [25]:
joined_features.read().show()

+---------+---+------+----------+-----+---------+--------+----+------+---+---+--------+---+------+----+
|     loan| id|hyears|commission|  car|   salary|  hvalue|year|elevel|age| id| zipcode| id| class|year|
+---------+---+------+----------+-----+---------+--------+----+------+---+---+--------+---+------+----+
| 564724.2|  1|    30|       0.0|car17|120499.73|205000.0|2020|level3| 54|  1|zipcode5|  1|groupA|2020|
| 216839.7|  6|    19|  52809.15| car7| 41365.43|135000.0|2020|level2| 71|  6|zipcode3|  6|groupA|2020|
|122025.08|  3|    22|       0.0| car1|119159.65|145000.0|2020|level2| 49|  3|zipcode4|  3|groupB|2020|
| 458679.8|  5|     1|       0.0|car15| 93956.32|135000.0|2020|level1| 59|  5|zipcode2|  5|groupA|2020|
| 99629.62|  4|    30|  52593.63| car9|  20000.0|185000.0|2020|level0| 56|  4|zipcode2|  4|groupB|2020|
|233216.06|  7|    23|       0.0|car17| 94805.61|135000.0|2020|level1| 32|  7|zipcode2|  7|groupA|2020|
|455015.34|  2|     2|       0.0|car10|160893.77|179000.0|2020|l